In [1]:
import numpy as np 
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
import torch
import time
import wandb
wandb.login(key="f659082c2b19bf3ffaaceceb36c1e280541f6b11")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: yeshu183 (yeshu183-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
base_path_train = "/kaggle/input/naturalist/inaturalist_12K/train" 
input_shape = (224,224)
id2label = {}
label2id = {}
label_list = []
X = []
y = []
for id,label in enumerate(os.listdir(base_path_train)):
    # if label[0]==".":
    #     continue
    id2label[id] = label
    label2id[label] = id
    label_list.append(label)
    label_path = os.path.join(base_path_train,label)
    for img_path in os.listdir(label_path):
        # if img_path == ".DS_Store":
        #     continue
        img = np.array(Image.open(os.path.join(label_path,img_path)))
        img.resize((*input_shape,3))
        X.append(img)
        y.append(id)
    print(f"id:{id}, Label: {label} done")
X = np.array(X).transpose(0, 3, 1, 2)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)
print(X.shape)
print(y.shape)

id:0, Label: Reptilia done
id:1, Label: Animalia done
id:2, Label: Arachnida done
id:3, Label: Amphibia done
id:4, Label: Aves done
id:5, Label: Mollusca done
id:6, Label: Fungi done
id:7, Label: Insecta done
id:8, Label: Plantae done
id:9, Label: Mammalia done
torch.Size([9999, 3, 224, 224])
torch.Size([9999])


In [ ]:
base_path_test = "/kaggle/input/naturalist/inaturalist_12K/val" 
input_shape = (224,224)
X_test = []
y_test = []
for id,label in enumerate(os.listdir(base_path_test)):
    # if label[0]==".":
    #     continue
    label_path = os.path.join(base_path_test,label)
    for img_path in os.listdir(label_path):
        # if img_path == ".DS_Store":
        #     continue
        img = np.array(Image.open(os.path.join(label_path,img_path)))
        img.resize((*input_shape,3))
        X_test.append(img)
        y_test.append(id)
    print(f"Test data id:{id}, Label: {label} done")
X_test = np.array(X_test).transpose(0, 3, 1, 2)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)
print(X_test.shape)
print(y_test.shape)

Test data id:0, Label: Reptilia done
Test data id:1, Label: Animalia done


In [ ]:
print(id2label)
print(label2id)
print(label_list)

## Part-B

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets, models, transforms
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=42)
# #Getting (X,y) pairs
# train_data = TensorDataset(X_train, y_train)
# val_data = TensorDataset(X_val, y_val)
# test_data = TensorDataset(X_test, y_test)

In [ ]:
import torch.nn as nn
import torchvision.models as models

class PretrainedCNN(nn.Module):
    def __init__(self, arch='efficientnet_v2_s', strategy=1, num_classes=10, dropout=0.5):
        super().__init__()
        self.arch = arch.lower()

        # Load pretrained model
        if self.arch == 'efficientnet_v2_s':
            self.model = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.DEFAULT)
            in_features = self.model.classifier[1].in_features
            self.model.classifier[1] = nn.Sequential(
                nn.Dropout(dropout),
                nn.Linear(in_features, num_classes)
            )
            self.feature_layers = self.model.features

        elif self.arch == 'resnet50':
            self.model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
            in_features = self.model.fc.in_features
            self.model.fc = nn.Linear(in_features, num_classes)
            # Combine all layer blocks into a list for uniform strategy application
            self.feature_layers = nn.Sequential(
                self.model.conv1,
                self.model.bn1,
                self.model.relu,
                self.model.maxpool,
                self.model.layer1,
                self.model.layer2,
                self.model.layer3,
                self.model.layer4,
            )

        elif self.arch == 'vgg16':
            self.model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
            in_features = self.model.classifier[6].in_features
            self.model.classifier[6] = nn.Linear(in_features, num_classes)
            self.feature_layers = self.model.features

        else:
            raise ValueError(f"Unsupported architecture: {arch}")

        # Apply freezing strategy
        self._freeze_strategy(strategy)

    def _freeze_strategy(self, strategy):
        # Freeze all blocks except the last `strategy` blocks
        total_blocks = len(list(self.feature_layers.children()))
        for i, child in enumerate(self.feature_layers.children()):
            if i < total_blocks - strategy:
                for param in child.parameters():
                    param.requires_grad = False

    def forward(self, x):
        return self.model(x)

In [ ]:
from torch.utils.data import Dataset

class CustomImageDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        img = self.X[idx]
        label = self.y[idx]

        # Convert tensor image to PIL for applying torchvision transforms
        img = transforms.ToPILImage()(img)

        if self.transform:
            img = self.transform(img)

        return img, label

In [ ]:
sweep_config = {
    'method': 'bayes',
    'name': 'pretrained_unfreeze_strategy_sweep',
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'arch': {
            'values': ['resnet50', 'vgg16', 'efficientnet_v2_s']
        },
        'strategy': {
            'values': [0, 1, 2, 3, 4]  # 0 = freeze all, 1+ = unfreeze last N blocks
        },
        'epochs': {
            'values': [5,10]
        },
        'batch_size':{'values': [32,64]},
        'learning_rate':{'values':[1e-3,1e-2]}
    }
}

In [10]:
def train_sweep():
    wandb.init()
    config = wandb.config

    run_name = f"{config.arch}_unfreeze_{config.strategy}"
    wandb.run.name = run_name

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = PretrainedCNN(
        arch=config.arch,
        strategy=config.strategy,
        num_classes=10,
    ).to(device)

    # Transforms
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    train_dataset = CustomImageDataset(X_train, y_train, transform=transform)
    val_dataset = CustomImageDataset(X_val, y_val, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=config.learning_rate)

    for epoch in range(config.epochs):
        epoch_start_time = time.time()
        model.train()
        running_loss, correct, total = 0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_acc = 100 * val_correct / val_total

        epoch_time = time.time() - epoch_start_time

        wandb.log({
            'train_loss': train_loss,
            'train_accuracy': train_acc,
            'val_loss': val_loss / len(val_loader),
            'val_accuracy': val_acc,
            'epoch_time': epoch_time
        })

        print(f"Epoch {epoch+1} | Time: {epoch_time:.2f}s | "
              f"Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss / len(val_loader):.3f}, Val Acc: {val_acc:.2f}%")

In [11]:
#model = PretrainedCNN(freeze_layers='all_but_last', num_classes=10, dropout=0.5).to(device)

In [12]:
wandb.finish()
sweep_id = wandb.sweep(sweep_config, project="DA6401_Assignment_02_B")
wandb.agent(sweep_id, function=train_sweep, count=50)  # Runs 50 experiments

Create sweep with ID: bwse0o6g
Sweep URL: https://wandb.ai/yeshu183-indian-institute-of-technology-madras/DA6401_Assignment_02_B/sweeps/bwse0o6g


wandb: Agent Starting Run: sdju2n07 with config:
wandb: 	arch: resnet50
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	strategy: 3
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 173MB/s] 


Epoch 1 | Time: 44.49s | Train Loss: 2.146, Train Acc: 22.88% | Val Loss: 2.265, Val Acc: 23.70%
Epoch 2 | Time: 43.55s | Train Loss: 2.038, Train Acc: 27.44% | Val Loss: 2.078, Val Acc: 27.00%
Epoch 3 | Time: 43.34s | Train Loss: 1.953, Train Acc: 31.13% | Val Loss: 2.298, Val Acc: 29.95%
Epoch 4 | Time: 43.30s | Train Loss: 1.878, Train Acc: 33.55% | Val Loss: 2.193, Val Acc: 28.40%
Epoch 5 | Time: 43.34s | Train Loss: 1.760, Train Acc: 38.27% | Val Loss: 2.585, Val Acc: 30.80%


epoch_time,█▂▁▁▁
train_accuracy,▁▃▅▆█
train_loss,█▆▅▃▁
val_accuracy,▁▄▇▆█
val_loss,▄▁▄▃█
epoch_time,43.33877
train_accuracy,38.26728
train_loss,1.75997
val_accuracy,30.8
val_loss,2.58486


...

Epoch 1 | Time: 46.00s | Train Loss: 2.171, Train Acc: 21.43% | Val Loss: 2.306, Val Acc: 24.30%
Epoch 2 | Time: 45.95s | Train Loss: 2.073, Train Acc: 26.48% | Val Loss: 2.100, Val Acc: 23.45%
Epoch 3 | Time: 45.98s | Train Loss: 2.009, Train Acc: 28.77% | Val Loss: 2.101, Val Acc: 27.20%
Epoch 4 | Time: 45.87s | Train Loss: 1.935, Train Acc: 31.95% | Val Loss: 2.033, Val Acc: 27.65%
Epoch 5 | Time: 45.96s | Train Loss: 1.858, Train Acc: 34.20% | Val Loss: 2.019, Val Acc: 30.55%


epoch_time,█▅▇▁▆
train_accuracy,▁▄▅▇█
train_loss,█▆▄▃▁
val_accuracy,▂▁▅▅█
val_loss,█▃▃▁▁
epoch_time,45.95845
train_accuracy,34.20428
train_loss,1.85804
val_accuracy,30.55
val_loss,2.01856


wandb: Agent Starting Run: ray7j3fn with config:
wandb: 	arch: resnet50
wandb: 	batch_size: 64
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	strategy: 4


Epoch 1 | Time: 45.78s | Train Loss: 2.165, Train Acc: 22.18% | Val Loss: 2.155, Val Acc: 22.30%
Epoch 2 | Time: 45.74s | Train Loss: 2.079, Train Acc: 26.02% | Val Loss: 2.041, Val Acc: 26.90%
Epoch 3 | Time: 45.77s | Train Loss: 1.996, Train Acc: 29.00% | Val Loss: 2.248, Val Acc: 28.20%
Epoch 4 | Time: 45.72s | Train Loss: 1.935, Train Acc: 31.79% | Val Loss: 2.062, Val Acc: 28.35%
Epoch 5 | Time: 45.79s | Train Loss: 1.856, Train Acc: 34.63% | Val Loss: 2.022, Val Acc: 29.10%


epoch_time,▇▄▆▁█
train_accuracy,▁▃▅▆█
train_loss,█▆▄▃▁
val_accuracy,▁▆▇▇█
val_loss,▅▂█▂▁
epoch_time,45.79366
train_accuracy,34.62933
train_loss,1.85609
val_accuracy,29.1
val_loss,2.02208


In [ ]:
from types import SimpleNamespace
def train_best_model(best_config,n_epochs=20):
    config = SimpleNamespace(**best_config)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = PretrainedCNN(
        arch=config.arch,
        strategy=config.strategy,
        num_classes=10,
    ).to(device)

    # Define transforms
    transform_train = transforms.Compose([
        transforms.Resize((224, 224)),  # Ensures all images are same size
        transforms.ToTensor()
    ])
    
    transform_val = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    # Use transformed datasets
    train_dataset = CustomImageDataset(X_train, y_train, transform=transform_train)
    val_dataset = CustomImageDataset(X_val, y_val, transform=transform_val)
    #test_dataset = CustomImageDataset(X_test, y_test, transform=transform_val)

    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)
    #test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(n_epochs):
        epoch_start_time = time.time()
        model.train()
        running_loss, correct, total = 0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_loss = running_loss / len(train_loader)
        train_acc = 100 * correct / total

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_acc = 100 * val_correct / val_total

        epoch_time = time.time() - epoch_start_time

        wandb.log({
            'train_loss': train_loss,
            'train_accuracy': train_acc,
            'val_loss': val_loss,
            'val_accuracy': val_acc,
            'epoch_time': epoch_time
        })

        print(f"Epoch {epoch+1} | Time: {epoch_time:.2f}s | "
              f"Train Loss: {train_loss:.3f}, Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.3f}, Val Acc: {val_acc:.2f}%")

In [ ]:
api = wandb.Api()

sweep_path = "DA6401_Assignment_02_B/pretrained_unfreeze_strategy_sweep"
runs = api.runs(sweep_path)

sorted_runs = sorted(runs, key=lambda run: run.summary.get("val_accuracy", 0), reverse=True)

best_run = sorted_runs[0]
best_config = {}
print(len(sorted_runs))
print("Best Run Name:", best_run.name)
print("Best Val Accuracy:", best_run.summary["val_accuracy"])
print("Best Hyperparameters:")
for k, v in best_run.config.items():
    best_config[k] = v
print(best_config)

In [ ]:
model = train_best_model(best_config,n_epochs=30)

In [ ]:
transform_test = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
test_dataset = CustomImageDataset(X_test, y_test, transform=transform_test)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)
model.eval()
test_loss, test_correct, test_total = 0, 0, 0
all_preds = []
all_imgs = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

        all_preds.extend(predicted.cpu())
        all_imgs.extend(inputs.cpu())

test_accuracy = 100 * test_correct / test_total
print(f" Final Test Accuracy: {test_accuracy:.2f}%")

In [ ]:
# import matplotlib.pyplot as plt

# epochs = range(1, len(train_losses) + 1)

# # Plot Loss
# plt.figure(figsize=(12, 5))
# plt.subplot(1, 2, 1)
# plt.plot(epochs, train_losses, label='Train Loss', marker='o')
# plt.plot(epochs, val_losses, label='Validation Loss', marker='o')
# plt.title('Loss over Epochs')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.grid(True)

# # Plot Accuracy
# plt.subplot(1, 2, 2)
# plt.plot(epochs, train_accuracies, label='Train Accuracy', marker='o')
# plt.plot(epochs, val_accuracies, label='Validation Accuracy', marker='o')
# plt.title('Accuracy over Epochs')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy (%)')
# plt.legend()
# plt.grid(True)

# plt.tight_layout()
# plt.show()